In [1]:
import os
import cv2
import random

from tqdm import tqdm
from common.tools import random_ps, random_crop

import pandas as pd
import numpy as np
import seaborn as sns
import albumentations as albu
import matplotlib.pyplot as plt

In [2]:
train_path = './data/train/img'
aux1_path = './data/aux_1/img'
test_path = './data/test/img'
aux2_paths = ['./data/aux_2/screenshot/img',
             './data/aux_2/document/img',
             './data/aux_2/cover/img']


train = os.listdir(train_path)
aux1 = os.listdir(aux1_path)
test = os.listdir(test_path)

In [9]:
ze_path = './data/ze'
ze_list = []
zes = os.listdir(ze_path)
for i in tqdm(zes):
    img_path = os.path.join(ze_path, i)
    ze_list.append(img_path)

ze_df = pd.DataFrame(ze_list, columns=['image'])
ze_df.to_csv('./data/ze.csv', sep=',', header=True, index=True, index_label='index')

100%|████████████████████████████████████████| 10/10 [00:00<00:00, 10583.66it/s]


In [6]:
path2608 = './data/2608/images'
mask_list = os.listdir(path2608)
mask_list.sort()
blank_list = []
f=open("0.txt","w+")
for item in tqdm(mask_list):
    mask_path = os.path.join(path2608, item)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    cnt = np.sum(mask > 0)
    if cnt < 10:
        f.write("%4s%10s\n"%(cnt, item))
f.close()

100%|██████████████████████████████████████| 4000/4000 [00:13<00:00, 303.42it/s]


In [15]:
aux_list = []

for aux2_path in aux2_paths:
    auxs = os.listdir(aux2_path)
    for i in tqdm(auxs):
        img_path = os.path.join(aux2_path, i)
        img = cv2.imread(img_path)
        mask_path = img_path.replace('img', 'mask').replace('.jpg', '.png')
        mask = np.zeros(img.shape[:2])
        cv2.imwrite(mask_path, mask.astype(np.uint8))
        max_pix = np.argmax(np.bincount(img.ravel()))
        aux_list.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png'), max_pix])

similar = []
for i in tqdm(aux_list):
    similar_list = []
    for j in aux_list:
        if i[2]==j[2]:
            similar_list.append(j[0])
    similar.append(random.choice(similar_list))

print(len(aux_list))
df1 = pd.DataFrame(aux_list, columns=['image', 'mask', 'max_pix'])
df1['similar'] = similar
df1.to_csv('./data/aux2.csv', sep=',', header=True, index=True, index_label='index')

100%|██████████████████████████████████████| 4609/4609 [00:07<00:00, 588.64it/s]


4609


In [16]:
df = pd.read_csv('./data/aux2.csv')
aux_list = []

for index, row in df.iterrows():
    n = 0
    while n < 3:
        loop = random.randint(1,4)
        image = cv2.imread(row['image'])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(row['mask'], cv2.IMREAD_GRAYSCALE) / 255

        for i in range(loop):
            image, mask = random_ps(image, row['similar'], mask)
        mask[mask > 0] = 255
        if image.shape[1] < 512 or image.shape[0] < 512:
            rate = random.uniform(1,2)
        elif image.shape[1] > 1024 or image.shape[0] > 1024:
            rate = random.uniform(0.5,1)
        else:
            rate = 1
        image, mask = cv2.resize(image, None, fx=rate, fy=rate), cv2.resize(mask, None, fx=rate, fy=rate)
        
        try:
            crop_h = random.randint(512, min(1024, image.shape[0]))
            crop_w = random.randint(512, min(1024, image.shape[1]))
            image, mask = random_crop(image, crop_h, crop_w), random_crop(mask, crop_h, crop_w)
        except:
            n += 1
            if n == 3:
                print(row['image'])
                # os.remove(row['image'])
                # os.remove(row['mask'])
                break
            continue
        aux_list.append([row['image'], row['mask']])
        cv2.imwrite(row['mask'], mask.astype(np.uint8))
        cv2.imwrite(row['image'], image.astype(np.uint8))
        break

aux_list = pd.DataFrame(aux_list, columns=['image', 'mask'])
aux_list.to_csv('./data/aux2.csv', sep=',', header=True, index=True, index_label='index')


















































































































./data/aux_2/screenshot/img/0000505.jpg









./data/aux_2/screenshot/img/0003069.jpg

















































































































































































































./data/aux_2/screenshot/img/0000723.jpg


















































./data/aux_2/screenshot/img/0002236.jpg































































































































































./data/aux_2/screenshot/img/0002758.jpg


























































./data/aux_2/screenshot/img/0001940.jpg








./data/aux_2/screenshot/img/0002133.jpg







































































































































./data/aux_2/screenshot/img/0001355.jpg









































































































































































































































./data/aux_2/screenshot/img/0001265.jpg


























































































./data/aux_2/screenshot/img/0001620.jpg
















































































































































































./data/aux_2/screenshot/img/0001807.jpg















./data/aux_2/screenshot/img/0001270.jpg



















































































































































































































































































































































































./data/aux_2/document/img/0000080.jpg

































./data/aux_2/document/img/0002183.jpg
































































































./data/aux_2/cover/img/0001496.jpg




















































































































































































































































































































































































































































































































































































































































































AttributeError: 'list' object has no attribute 'to_csv'

In [19]:
aux_list = []
df = pd.read_csv('./data/aux2.csv')

for index, row in df.iterrows():
    if row['image'].endswith('jpg'):
        img = cv2.imread(row['image']).ravel()
        max_pix = np.argmax(np.bincount(img))
        aux_list.append([row['image'], row['mask'], max_pix])
        
        
similar = []
for i in tqdm(aux_list):
    similar_list = []
    for j in aux_list:
        if i[2]==j[2]:
            similar_list.append(j[0])
    similar.append(random.choice(similar_list))


print(len(aux_list))
df1 = pd.DataFrame(aux_list, columns=['image', 'mask', 'max_pix'])
df1['similar'] = similar
df1.to_csv('./data/preprocessed_aux_2.csv', sep=',', header=True, index=True, index_label='index')

100%|█████████████████████████████████████| 4466/4466 [00:02<00:00, 1644.26it/s]

4466


only collect train and test dataset

In [4]:
train_list = []
test_list = []

for i in tqdm(train):
    if i.endswith('jpg'):
        img_path = os.path.join(train_path, i)
        img = cv2.imread(img_path).ravel()
        max_pix = np.argmax(np.bincount(img))
        train_list.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png'), max_pix])
        
for i in tqdm(aux1):
    if i.endswith('jpg'):
        img_path = os.path.join(aux1_path, i)
        img = cv2.imread(img_path).ravel()
        max_pix = np.argmax(np.bincount(img))
        train_list.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png'), max_pix])

similar = []
for i in tqdm(train_list):
    similar_list = []
    for j in train_list:
        if i[2]==j[2]:
            similar_list.append(j[0])
    similar.append(random.choice(similar_list))


print(len(train_list))
df1 = pd.DataFrame(train_list, columns=['image', 'mask', 'max_pix'])
df1['similar'] = similar
df1.to_csv('./data/train_data.csv', sep=',', header=True, index=True, index_label='index')


for i in tqdm(test):
    if i.endswith('jpg'):
        img_path = os.path.join(test_path, i)
        test_list.append(img_path)

print(len(test_list))
df2 = pd.DataFrame(test_list, columns=['image'])
df2.to_csv('./data/test_data.csv', sep=',', header=True, index=True, index_label='index')


100%|███████████████████████████████████| 4000/4000 [00:00<00:00, 594283.45it/s]

6005
4000


classify train dataset as size

In [3]:
list1 = []
list2 = []
list3 = []
list4 = []

for i in tqdm(train):
    if i.endswith('jpg'):
        img_path = os.path.join(train_path, i)
        w, h = cv2.imread(img_path).shape[:2]
        if w < 768 and h < 768:
            list1.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png')])
        elif w < 768 and h >= 768:
            list2.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png')])
        elif w >= 768 and h < 768:
            list3.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png')])
        elif w >= 768 and h >= 768:
            list4.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png')])

for i in tqdm(aux1):
    if i.endswith('jpg'):
        img_path = os.path.join(aux1_path, i)
        w, h = cv2.imread(img_path).shape[:2]
        if w < 768 and h < 768:
            list1.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png')])
        elif w < 768 and h >= 768:
            list2.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png')])
        elif w >= 768 and h < 768:
            list3.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png')])
        elif w >= 768 and h >= 768:
            list4.append([img_path, img_path.replace('img', 'mask').replace('.jpg', '.png')])

100%|██████████████████████████████████████| 2005/2005 [00:19<00:00, 101.05it/s]


In [4]:
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))

df1 = pd.DataFrame(list1, columns=['image','mask'])
df2 = pd.DataFrame(list2, columns=['image','mask'])
df3 = pd.DataFrame(list3, columns=['image','mask'])
df4 = pd.DataFrame(list4, columns=['image','mask'])

df1.to_csv('./data/train_for_11.csv', sep=',', header=True, index=True, index_label='index')
df2.to_csv('./data/train_for_12.csv', sep=',', header=True, index=True, index_label='index')
df3.to_csv('./data/train_for_21.csv', sep=',', header=True, index=True, index_label='index')
df4.to_csv('./data/train_for_22.csv', sep=',', header=True, index=True, index_label='index')

1915
964
1034
2092


classify test dataset as size

In [5]:
list1 = []
list2 = []
list3 = []
list4 = []

for i in tqdm(test):
    if i.endswith('jpg'):
        img_path = os.path.join(test_path, i)
        w, h = cv2.imread(img_path).shape[:2]
        if w < 768 and h < 768:
            list1.append(img_path)
        elif w < 768 and h >= 768:
            list2.append(img_path)
        elif w >= 768 and h < 768:
            list3.append(img_path)
        elif w >= 768 and h >= 768:
            list4.append(img_path)

100%|██████████████████████████████████████| 4000/4000 [00:24<00:00, 163.11it/s]


In [6]:
print(len(list1))
print(len(list2))
print(len(list3))
print(len(list4))

df1 = pd.DataFrame(list1, columns=['image'])
df2 = pd.DataFrame(list2, columns=['image'])
df3 = pd.DataFrame(list3, columns=['image'])
df4 = pd.DataFrame(list4, columns=['image'])

df1.to_csv('./data/test_for_11.csv', sep=',', header=True, index=True, index_label='index')
df2.to_csv('./data/test_for_12.csv', sep=',', header=True, index=True, index_label='index')
df3.to_csv('./data/test_for_21.csv', sep=',', header=True, index=True, index_label='index')
df4.to_csv('./data/test_for_22.csv', sep=',', header=True, index=True, index_label='index')

1105
1046
959
890
